In [2]:
from dlcliche.notebook import *
from dlcliche.utils import *
sys.path.append('..')
import img_hash
from utils import *

opt = Options(("--dataroot /data/mvtec_ad/original "
               "--project mvtecad_phash "
               "--suffix .png "
              )).parse()


params = EasyDict()
params.suffix = opt.suffix
params.project = opt.project
params.work_folder = 'tmp'
params.hash_algorithm = 'phash'
params.hash_size = 32

----------------- Options ---------------
             anomaly_gray: False                         
            app_album_tfm: none                          
        app_dataset_class: default                       
                 backbone: resnet34                      
               batch_size: 16                            
                crop_size: 224                           	[default: 256]
                 dataroot: /data/mvtec_ad/original       	[default: None]
                  gpu_ids: 0                             
                load_size: 256                           	[default: 286]
                       lr: 0.0003                        	[default: 0.001]
                  lr_step: 10                            
                    model: arc_face                      
                 n_epochs: 50                            	[default: 30]
                  no_norm: False                         
              num_threads: 12                            	[default: 4

In [8]:
from utils import evaluate_MVTecAD

results = evaluate_MVTecAD(opt.dataroot, img_hash.ImgHashAnoDet, params,
                test_targets=None
                )

Evaluating: ['bottle', 'cable', 'capsule', 'carpet', 'grid', 'hazelnut', 'leather', 'metal_nut', 'pill', 'screw', 'tile', 'toothbrush', 'transistor', 'wood', 'zipper']

--- Start evaluating [bottle] ----
 preprocessing...
 training...
 evaluating...
Results: {'target': ['bottle'], 'auc': [0.769047619047619]}

--- Start evaluating [cable] ----
 preprocessing...
 training...
 evaluating...
Results: {'target': ['cable'], 'auc': [0.717859820089955]}

--- Start evaluating [capsule] ----
 preprocessing...
 training...
 evaluating...
Results: {'target': ['capsule'], 'auc': [0.7640606302353412]}

--- Start evaluating [carpet] ----
 preprocessing...
 training...
 evaluating...
Results: {'target': ['carpet'], 'auc': [0.476123595505618]}

--- Start evaluating [grid] ----
 preprocessing...
 training...
 evaluating...
Results: {'target': ['grid'], 'auc': [0.8391812865497076]}

--- Start evaluating [hazelnut] ----
 preprocessing...
 training...
 evaluating...
Results: {'target': ['hazelnut'], 'auc':

In [12]:
results.to_csv(f'sample_{opt.project}.csv')
results

,target,auc
0,bottle,0.769048
1,cable,0.717860
2,capsule,0.764061
3,carpet,0.476124
4,grid,0.839181
5,hazelnut,0.836250
6,leather,0.897079
7,metal_nut,0.597996
8,pill,0.628205
9,screw,0.686719


## If you want to try one by one

In [10]:
from dlcliche.utils import (load_pkl, save_as_pkl_binary)
sys.path.append('..')
from base_ano_det import BaseAnoDet


test_target = 'transistor'
folder = Path(opt.dataroot)/test_target
good_files = sorted(folder.glob(f'train/good/*{params.suffix}'))
test_files = sorted(folder.glob(f'test/*/*{params.suffix}'))
test_y_trues = [f.parent.name != 'good' for f in test_files]

det = img_hash.ImgHashAnoDet(params)
det.prepare_experiment()
# no need: det.setup_train(good_files)
det.train_model(good_files)

det.predict(test_files[0])

0.265625

## If you run for many parameter sets, summarize...

In [25]:
def read_result(file_name, title):
    df = pd.read_csv(file_name)[['target', 'auc']].set_index('target')
    df.columns = [title]
    return df

dfs = [read_result(f, title) for f, title in [
    ('sample_mvtecad_phash16.csv', 'phash=16'),
    ('sample_mvtecad_phash.csv', 'phash=32'),
    ('sample_mvtecad_phash64.csv', 'phash=64'),
    ('sample_mvtecad_whash16.csv', 'whash=16'),
    ('sample_mvtecad_whash.csv', 'whash=32'),
]]

df = pd.concat(dfs, axis=1)
df

,phash=16,phash=32,phash=64,whash=16,whash=32
target,,,,,
bottle,0.894048,0.769048,0.834921,0.831746,0.835714
cable,0.763681,0.717860,0.655454,0.688999,0.756747
capsule,0.732948,0.764061,0.751296,0.540088,0.713801
carpet,0.549559,0.476124,0.493780,0.600120,0.461276
grid,0.913952,0.839181,0.786967,0.619883,0.563910
hazelnut,0.786250,0.836250,0.812500,0.462679,0.530714
leather,0.946671,0.897079,0.757812,0.838655,0.793648
metal_nut,0.660557,0.597996,0.556452,0.714809,0.782014
pill,0.649345,0.628205,0.635706,0.648663,0.854883


In [26]:
print(df.to_markdown())

| target     |   phash=16 |   phash=32 |   phash=64 |   whash=16 |   whash=32 |
|:-----------|-----------:|-----------:|-----------:|-----------:|-----------:|
| bottle     |   0.894048 |   0.769048 |   0.834921 |   0.831746 |   0.835714 |
| cable      |   0.763681 |   0.71786  |   0.655454 |   0.688999 |   0.756747 |
| capsule    |   0.732948 |   0.764061 |   0.751296 |   0.540088 |   0.713801 |
| carpet     |   0.549559 |   0.476124 |   0.49378  |   0.60012  |   0.461276 |
| grid       |   0.913952 |   0.839181 |   0.786967 |   0.619883 |   0.56391  |
| hazelnut   |   0.78625  |   0.83625  |   0.8125   |   0.462679 |   0.530714 |
| leather    |   0.946671 |   0.897079 |   0.757812 |   0.838655 |   0.793648 |
| metal_nut  |   0.660557 |   0.597996 |   0.556452 |   0.714809 |   0.782014 |
| pill       |   0.649345 |   0.628205 |   0.635706 |   0.648663 |   0.854883 |
| screw      |   0.598176 |   0.686719 |   0.647469 |   0.573786 |   0.605759 |
| tile       |   0.440657 |   0.478896 |